<a href="https://colab.research.google.com/github/mdahfienon/MachineLearningAssigment/blob/main/svm_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## First 2 question

In [1]:
import pandas as pd
from sklearn.svm import SVC

In [2]:
path = "/content/drive/MyDrive/MachineLearning/Classification et Evaluation/SupportVectorMachines/"
train_df = pd.read_csv(path + "DiabetesTrain.csv", index_col=0)
test_df = pd.read_csv(path + "DiabetesTest.csv", index_col=0)

col = train_df.columns

In [3]:
# target et predictors
X_train = train_df[train_df.columns[:-1]]
Y_train = train_df[train_df.columns[-1]]

X_test = test_df[test_df.columns[:-1]]
Y_test = test_df[test_df.columns[-1]]

In [4]:
# creating the fold
# folds = {}
# k = 0
# for i in range(1,1000, 100):
#     folds["fold_" + str(k+1)] = train_df.loc[i:i+99,]
#     k +=1

# col = train_df.columns

Question 1

In [5]:
# creating the 10-fold

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
folds = KFold(n_splits=10)


In [6]:
# base model
basicModel = SVC(kernel = "linear") 
scores = cross_val_score(basicModel, X_train, Y_train, scoring='accuracy',
                         cv=folds)
scores.mean()

0.717

Question 2

In [7]:
accur = []
for i in range(11):
    polyModel = SVC(kernel = "poly", degree=i).fit(X_train, Y_train)
    accur.append( str(i)+ ":" + str(polyModel.score(X_train, Y_train)))
accur

['0:0.535',
 '1:0.72',
 '2:0.731',
 '3:0.737',
 '4:0.73',
 '5:0.725',
 '6:0.723',
 '7:0.713',
 '8:0.71',
 '9:0.715',
 '10:0.721']

In [8]:
folds = KFold(n_splits=10)
polyModel = SVC(kernel = "poly", degree=2) 
scores = cross_val_score(polyModel, X_train, Y_train, scoring='accuracy',
                         cv=folds)
scores.mean()

0.7229999999999999

## preprocessing the data

In [9]:
# from sklearn.preprocessing import KBinsDiscretizer as kbd

# Discretize BMI
train_df_prepro = train_df.copy()
train_df_prepro["BMI_cat"] = pd.cut(train_df_prepro["BMI"], 
                             [0, 18.6, 25, 30, 1000], 
                             labels = ["underweight", "healthy", "overweight", "obesity" ]) 

train_df_prepro[["BMI", "BMI_cat"]]


,BMI,BMI_cat
1,33.0,obesity
2,21.0,healthy
3,32.0,obesity
4,35.0,obesity
5,27.0,overweight
...,...,...
996,44.0,obesity
997,33.0,obesity
998,41.0,obesity
999,34.0,obesity


In [10]:
# import seaborn as sns
# sns.histplot(x = "PhysHlth", data = train_df_prepro)
# sns.histplot(x = "MentHlth", data = train_df_prepro)

In [11]:
# perform mutual information
from sklearn.feature_selection import mutual_info_classif as mi
infoGive = mi(train_df[col[:-1]], train_df[col[-1]])
mi_analysis = pd.Series(infoGive)
mi_analysis.index = col[:-1]
mi_analysis.sort_values(ascending = False)
goodcol = mi_analysis.sort_values(ascending = False).index[:-5]
X_train_prepro = train_df[goodcol]
proc_train_df = X_train_prepro.join(Y_train)
proc_X_train = proc_train_df[proc_train_df.columns[:-1]]
proc_Y_train = proc_train_df[proc_train_df.columns[-1]]

## Looking for best metaparameters for poly kernel

In [ ]:
from sklearn.model_selection import GridSearchCV

parametr = {
    "gamma" : ["scale", "auto"],
    "coef0" : [0.6, 0.9],
    "C" : [1,2,3]
}
model = SVC(kernel="poly", degree = 2)
mysearch = GridSearchCV(estimator = model, param_grid=parametr,
                        scoring = "accuracy", cv = folds)

mysearch.fit(proc_X_train, proc_Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=SVC(degree=2, kernel='poly'),
             param_grid={'C': [1, 2, 3], 'coef0': [0.6, 0.9],
                         'gamma': ['scale', 'auto']},
             scoring='accuracy')

In [ ]:
mysearch.best_estimator_

SVC(C=1, coef0=0.6, degree=2, kernel='poly')

In [16]:
bestPoly = SVC(C = 1, coef0=0.6, degree=2, kernel='poly')

scores = cross_val_score(bestPoly, proc_X_train, proc_Y_train, scoring='accuracy',
                         cv=folds)
scores.mean()



0.7370000000000001

In [17]:
scoresN = cross_val_score(bestPoly, X_train, Y_train, scoring='accuracy',
                         cv=folds)
scoresN.mean()

0.733

In [ ]:
# best poly model 

# SVC(kernel = "poly", degree = 2, gamma = "scale", coef0 = 5)

### RBF

In [ ]:
rbf_param = {
    "gamma" : ["scale", "auto", 0.8],
    "coef0" : [0.4, 0.6, 0.9],
    "C" : [1, 1.5, 3]
}

rbfModel = SVC(kernel="rbf")
myRbfsearch = GridSearchCV(estimator = rbfModel, param_grid=rbf_param,
                        scoring = "accuracy", cv = folds)

myRbfsearch.fit(proc_X_train, proc_Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=SVC(),
             param_grid={'C': [1, 1.5, 3], 'coef0': [0.4, 0.6, 0.9],
                         'gamma': ['scale', 'auto', 0.8]},
             scoring='accuracy')

In [ ]:
myRbfsearch.best_estimator_

SVC(C=3, coef0=0.4)

In [ ]:
bestrbf = SVC(C=3, kernel = "rbf", coef0=0.4)

rbf_scores = cross_val_score(bestrbf, proc_X_train, proc_Y_train, scoring='accuracy',
                         cv=folds)
rbf_scores.mean()

0.733

In [ ]:
rbf_scores2 = cross_val_score(bestrbf, X_train, Y_train, scoring='accuracy',
                         cv=folds)
rbf_scores2.mean()

0.724

###sigmoid kernel

In [ ]:
sig_param = {
    "gamma" : ["scale", "auto"],
    "coef0" : [0.4, 0.6, 0.9],
    "C" : [1, 1.5, 3, 4]
}

sigModel = SVC(kernel="sigmoid")
mySigsearch = GridSearchCV(estimator = sigModel, param_grid=sig_param,
                        scoring = "accuracy", cv = folds)

mySigsearch.fit(proc_X_train, proc_Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=SVC(kernel='sigmoid'),
             param_grid={'C': [1, 1.5, 3, 4], 'coef0': [0.4, 0.6, 0.9],
                         'gamma': ['scale', 'auto']},
             scoring='accuracy')

In [ ]:
mySigsearch.best_estimator_

SVC(C=3, coef0=0.4, kernel='sigmoid')

In [ ]:
bestsig = SVC(C=3, kernel = "sigmoid", coef0=0.4)

sig_scores = cross_val_score(bestsig, proc_X_train, proc_Y_train, scoring='accuracy',
                         cv=folds)
sig_scores.mean()

0.682

In [ ]:
sig_scoresN = cross_val_score(bestsig, X_train, Y_train, scoring='accuracy',
                         cv=folds)
sig_scoresN.mean()

0.688

## performance on test set

In [20]:
colRetained  = proc_train_df.columns

proc_test_df = test_df[colRetained]
proc_X_test = proc_test_df[colRetained[:-1]]
proc_Y_test = proc_test_df[colRetained[-1]]

In [21]:
my_poly_svm = SVC(C = 1, coef0=0.6, degree=2, kernel='poly').fit(proc_X_train, proc_Y_train)

my_rbf_svm = SVC(C=3, kernel = "rbf", coef0 = 0.4).fit(proc_X_train, proc_Y_train)
 
my_sigm_svm = SVC(C=3, kernel = "sigmoid", coef0 = 0.4).fit(proc_X_train, proc_Y_train)

mu_linear_svm = SVC(kernel = "linear").fit(proc_X_train, proc_Y_train)


In [22]:
[mu_linear_svm.score(proc_X_test,proc_Y_test),
 my_poly_svm.score(proc_X_test,proc_Y_test),
 my_rbf_svm.score(proc_X_test,proc_Y_test),
 my_sigm_svm.score(proc_X_test,proc_Y_test)]

[0.795, 0.77, 0.755, 0.73]

In [23]:
mu_linear_svm.coef_

array([[ 0.46110556,  0.7858873 ,  0.11008081,  0.07738337,  0.04733185,
        -0.00280587,  0.37859069,  0.40883022,  0.20998905, -0.04675902,
        -0.33502726, -0.09430243]])

In [24]:
proc_X_test.columns

Index(['GenHlth', 'HighBP', 'Age', 'DiffWalk', 'BMI', 'PhysHlth', 'HighChol',
       'HeartDiseaseorAttack', 'Stroke', 'PhysActivity', 'HvyAlcoholConsump',
       'Sex'],
      dtype='object')